In [ ]:
from agents import Agent, Runner
import os
from cloud_models import env
import asyncio

os.environ["OPENAI_API_KEY"] = env.OPENAI_API_KEY

In [40]:
from agents import ModelSettings


spanish_agent = Agent(
    name="Spanish agent",
    handoff_description="Agent that speaks only in Spanish",
    instructions="You only speak Spanish.",
)

english_agent = Agent(
    name="English agent",
    handoff_description="Agent that speaks only in English",
    instructions="You only speak English",
)

triage_agent = Agent(
    name="Triage agent",
    instructions="""
1. FIRST, write a one-sentence "Reasoning" block analyzing the language of the request.
2. SECOND, announce the connection to the user.
3. THIRD, execute the transfer tool.""",
    handoffs=[spanish_agent, english_agent],
    model="gpt-5-nano",
    model_settings=ModelSettings(
        extra_args={"service_tier": "flex"},
    ),
)


async def main():
    result = await Runner.run(triage_agent, input="what is 2 + 2?")
    print(result.final_output)
    return result


result = await main()

The answer to 2 + 2 is 4.


In [41]:
# This will print every message from every agent in the chain
for turn in result.raw_responses:
    # Most agents put their text in the 'content' field
    content = turn.output
    if content:
        print(f"Agent says: {content}")

Agent says: [ResponseReasoningItem(id='rs_03a644e346221dbf006971976add6481999561805b9ca60461', summary=[], type='reasoning', content=None, encrypted_content=None, status=None), ResponseFunctionToolCall(arguments='{}', call_id='call_InveerE7Fp5lg4ZKn1CjM2CB', name='transfer_to_english_agent', type='function_call', id='fc_03a644e346221dbf0069719770d6bc81999ae198d5a904298d', status='completed')]
Agent says: [ResponseOutputMessage(id='msg_03a644e346221dbf0069719771f85c81999f07458fc79fa45b', content=[ResponseOutputText(annotations=[], text='The answer to 2 + 2 is 4.', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')]


In [42]:
import json

# 1. Convert the Pydantic objects into a list of DICTIONARIES first
# Do NOT use .model_dump_json() here
data_as_dicts = [item.model_dump() for item in result.raw_responses[0].output]

# 2. Now use json.dumps on that list of dictionaries
final_json = json.dumps(data_as_dicts, indent=2)

# 3. Print the result
print(final_json)

[
  {
    "id": "rs_03a644e346221dbf006971976add6481999561805b9ca60461",
    "summary": [],
    "type": "reasoning",
    "content": null,
    "encrypted_content": null,
    "status": null
  },
  {
    "arguments": "{}",
    "call_id": "call_InveerE7Fp5lg4ZKn1CjM2CB",
    "name": "transfer_to_english_agent",
    "type": "function_call",
    "id": "fc_03a644e346221dbf0069719770d6bc81999ae198d5a904298d",
    "status": "completed"
  }
]
